[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/file/d/1zU3Cja1abmD54CTluirBnr5I5Sl8GYPx/view?usp=sharing)

Since the model is devoleped using tf version 1 we are installing the version tf.1.xx

In [ ]:
!pip install tensorflow-gpu==1.15.0

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
#!gdown --id 1HH23gWovcBwRQTh0kOwYIl55I-P0LtIY
# !gdown --id 1En86LcxRw701XIXOy31wHx4NTUTluYzY
!gdown --id 1oEVk04zVpQkvW9YdgRQ9k7tuMPvbEbLq

Downloading...
From: https://drive.google.com/uc?id=1oEVk04zVpQkvW9YdgRQ9k7tuMPvbEbLq
To: /content/FullData.zip
100% 508M/508M [00:07<00:00, 70.9MB/s]


In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/NVIDIA/OpenSeq2Seq.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q --depth 1 {git_repo_url}
  !pip uninstall -y -q pymc3
  !pip install --upgrade joblib
  !pip install -q youtube-dl librosa python_speech_features sentencepiece
  
  # create eval config
  !cp {project_name}/example_configs/speech2text/jasper10x5_LibriSpeech_nvgrad.py {project_name}/conf.py
  !sed -i -e 's/\/data\/librispeech\/librivox-test-clean/test/' {project_name}/conf.py
  !echo 'backend = "librosa"' >> {project_name}/conf.py 
  # !echo "wav_filename, wav_filesize, transcript" > {project_name}/test.csv
  # !echo "test.wav, UNUSED, UNUSED" >> {project_name}/test.csv
  
import sys
sys.path.append(project_name)

     |████████████████████████████████| 1.9 MB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 37.7 MB/s 


In [ ]:
!unzip /content/FullData.zip

In [ ]:
import pandas as pd

In [ ]:
df_testu = pd.read_csv("/content/FullData/test.csv")

In [ ]:
df_testu

,wav_filename,wav_filesize,transcript
0,file_00.wav,UNUSED,UNUSED
1,file_01.wav,UNUSED,UNUSED
2,file_010.wav,UNUSED,UNUSED
3,file_0100.wav,UNUSED,UNUSED
4,file_01000.wav,UNUSED,UNUSED
...,...,...,...
2615,file_0995.wav,UNUSED,UNUSED
2616,file_0996.wav,UNUSED,UNUSED
2617,file_0997.wav,UNUSED,UNUSED
2618,file_0998.wav,UNUSED,UNUSED


In [ ]:
for i in range(df_testu.shape[0]):
  path = "FullData/audio/"
  name = df_testu["wav_filename"][i]
  new_path = os.path.join(path, name)
  df_testu["wav_filename"][i] = new_path

In [ ]:
df_testu

,wav_filename,wav_filesize,transcript
0,FullData/audio/file_00.wav,UNUSED,UNUSED
1,FullData/audio/file_01.wav,UNUSED,UNUSED
2,FullData/audio/file_010.wav,UNUSED,UNUSED
3,FullData/audio/file_0100.wav,UNUSED,UNUSED
4,FullData/audio/file_01000.wav,UNUSED,UNUSED
...,...,...,...
2615,FullData/audio/file_0995.wav,UNUSED,UNUSED
2616,FullData/audio/file_0996.wav,UNUSED,UNUSED
2617,FullData/audio/file_0997.wav,UNUSED,UNUSED
2618,FullData/audio/file_0998.wav,UNUSED,UNUSED


In [ ]:
df_testu.to_csv("/content/OpenSeq2Seq/test.csv")

In [ ]:
!cp -r /content/FullData /content/OpenSeq2Seq

## Download pre-trained Model

In [ ]:
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id={file_id}" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm={confirm_text}&id={file_id}" -o {file_name}
  
if not exists(join(project_name, 'w2l_log_folder')):
  download_from_google_drive('1gzGT8HoVNKY1i5HNQTKaSoCu7JHV4siR', 'jasper_10x5_dr_sp_nvgrad.zip')
  !unzip jasper_10x5_dr_sp_nvgrad.zip
  !mv checkpoint {project_name}/jasper_log_folder

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   2372      0 --:--:-- --:--:-- --:--:--  2372
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2960M  100 2960M    0     0  73.7M      0  0:00:40  0:00:40 --:--:-- 41.0M
Archive:  jasper_10x5_dr_sp_nvgrad.zip
   creating: checkpoint/
  inflating: checkpoint/checkpoint   
  inflating: checkpoint/model.ckpt-439200.data-00000-of-00001  
  inflating: checkpoint/model.ckpt-439200.index  
  inflating: checkpoint/model.ckpt-439200.meta  


In [ ]:
!cd {project_name} && python run.py --config_file conf.py --mode=infer --infer_output_file=LMFull.txt --use_horovod=False --num_gpus=1 --batch_size_per_gpu 1












The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



*** Restoring from the latest checkpoint
*** Loading model from jasper_log_folder/model.ckpt-439200
*** Inference config:
{'batch_size_per_gpu': 1,
 'data_layer': <class 'open_seq2seq.data.speech2text.speech2text.Speech2TextDataLayer'>,
 'data_layer_params': {'backend': 'librosa',
                       'dataset_files': ['test.csv'],
                       'dither': 1e-05,
                       'input_type': 'logfbank',
                       'norm_per_feature': True,
                       'num_audio_features': 64,
                       'pad_to': 16,
                       'precompute_mel_basis': True,
          

In [ ]:
!cat '/content/OpenSeq2Seq/LMFull.txt'

In [ ]:
!pip install jiwer

     |████████████████████████████████| 50 kB 2.9 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149873 sha256=9838bb7ef9492d72bd8d0fdc5cbcfd00ffd0072daac38be2c59e052a0df1ac47
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
file2 = open("/content/OpenSeq2Seq/LMFull.txt","r+") 
file2.readline(0)
X2 = file2.readlines()
file2.close()

In [ ]:
NoLMPredictions = list(X2)

In [ ]:
len(NoLMPredictions)

2621

In [ ]:
import pandas as pd

In [ ]:
actual_text = pd.read_csv('/content/OpenSeq2Seq/FullData/text.csv')

In [ ]:
!pip install jiwer

     |████████████████████████████████| 50 kB 3.8 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149861 sha256=1e81e8e179e4e1279d09cea900d35fb2e67a8e400c7afcf63ac2358ffcb0a751
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
from jiwer import wer

In [ ]:
NoLMPredictions[0:5]

['wav_filename,predicted_transcript\n',
 'FullData/audio/file_00.wav,he hoped there would be stew for dinner turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick peppered flower fattened sauce\n',
 'FullData/audio/file_01.wav,stuff it into you his belly counselled him\n',
 'FullData/audio/file_010.wav,well now ennis i declare you have a head and so has my stick\n',
 'FullData/audio/file_0100.wav,in both these high mythical subjects the surrounding nature though suffering is still dignified and beautiful\n']

In [ ]:
NoLMPredictions.remove("wav_filename,predicted_transcript\n")

In [ ]:
NoLMPredictions

In [ ]:
NoLMPredictions[0][len(NoLMPredictions[0].split(",")[0])+1:-1]

'he hoped there would be stew for dinner turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick peppered flower fattened sauce'

In [ ]:
NoLMprediction_list = []
for i in range(len(NoLMPredictions)):
  temp = NoLMPredictions[i][len(NoLMPredictions[i].split(",")[0])+1:-1]
  NoLMprediction_list.append(temp)

In [ ]:
NoLMprediction_list

In [ ]:
NoLMwer_list = []
for i in range(len(NoLMPredictions)):
  indexu = int(os.path.basename(NoLMPredictions[i].split(",")[0])[6:-4])
  wer_temp2 = wer(actual_text['text'][indexu].lower(),NoLMprediction_list[i])
  NoLMwer_list.append(wer_temp2)

In [ ]:
NoLMwer_list

In [ ]:
Avg_wer = sum(NoLMwer_list) / len(NoLMwer_list)
Avg_wer

0.04387671045989565